In [1]:
#!pip install git+https://github.com/pmdevita/faster-whisper-live.git

In [11]:
#!pip install ipywidgets ipywebrtc faster-whisper-live

In [12]:
#!pip install ffmpeg-python  # 如果还没装 ffmpeg wrapper

In [1]:
!ffmpeg -f avfoundation -list_devices true -i ""

# 选择 MBP Microphone 的值，填到下一个格子里

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

In [13]:
import subprocess
import asyncio, threading
from faster_whisper_live import LiveWhisper

from ipywidgets import Output
out = Output()
display(out)

model = LiveWhisper(
    "small",               # 模型大小：tiny/base/small/medium/large
    device="cpu",          # CPU 也能跑得很快，或切换到 cuda
    compute_type="int8"    # int8 量化加速
)



# 用 FFmpeg 从系统麦克风采集 16kHz、单声道、16-bit PCM
#    macOS: avfoundation，Linux 可能用 alsa 或 pulse
ffmpeg_cmd = [
    "ffmpeg",
    "-f", "avfoundation",
    "-i", ":1",          # 注意这里是 ":0" 而不是 "0"
    "-ac", "1",          # 单声道
    "-ar", "16000",      # 16kHz
    "-f", "s16le",       # 原始 PCM 输出
    "-"
]

proc = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE)

def run_live():
    with out:
        for seg in model.transcribe(proc.stdout, decode_audio=False, vad_filter=True):
            print(f"[{seg.start:.2f}s → {seg.end:.2f}s] {seg.text}")

threading.Thread(target=run_live, daemon=True).start()

Output()

OMP: Warning #191: Forking a process while a parallel region is active is potentially unsafe.


In [1]:
import subprocess, threading
from ipywidgets import Output
from IPython.display import display
from faster_whisper_live import LiveWhisper

# —— 前端输出区 —— 
out = Output(layout={"border":"1px solid #ccc","height":"400px","overflow":"auto"})
display(out)

# —— 初始化模型 —— 
model = LiveWhisper("small", device="cpu", compute_type="int8")

# —— 启动 FFmpeg 采麦克风 —— 
ffmpeg_cmd = [
    "ffmpeg",
    "-loglevel", "quiet",
    "-f", "avfoundation",
    "-i", ":1",       # ← 用你的音频设备号
    "-ac", "1",
    "-ar", "16000",
    "-f", "s16le",
    "-"
]
proc = subprocess.Popen(ffmpeg_cmd, stdout=subprocess.PIPE)

# —— 后台线程：一边喂流，一边打印 —— 
def run_live():
    with out:
        for seg in model.transcribe(proc.stdout,
                                    decode_audio=False,
                                    vad_filter=True):
            print(f"[{seg.start:.2f}s → {seg.end:.2f}s] {seg.text}")

threading.Thread(target=run_live, daemon=True).start()

Output(layout=Layout(border_bottom='1px solid #ccc', border_left='1px solid #ccc', border_right='1px solid #cc…